In [3]:
from illufly.base.base_call import BaseCall

class BothHandler(BaseCall):
    def __init__(self):
        super().__init__()
        self.register_method(
            "handle",
            sync_handle=self.handle_request,
            async_handle=self.async_handle_request
        )        
    def handle_request(self, value):
        return f"sync: {value}"
    async def async_handle_request(self, value):
        return f"async: {value}"

handler = BothHandler()

# 同步调用
print("同步调用:")
result = handler.sync_method("handle", "test")
print(f"Result: {result}")

# 直接使用 await
print("\n直接使用await：")
await handler.async_method("handle", "test")

同步调用:
Result: sync: test

直接使用await：


'async: test'

In [7]:
from illufly.base.base_service import BaseService
from illufly.mq.message_bus import MessageBus

class MyService(BaseService):
    """用于测试的服务类"""
    def __init__(self, service_name: str = "test_service", message_bus_address: str = None):
        super().__init__(service_name, message_bus_address)
        self.register_method("server", async_handle=self._async_handler)
    
    async def _async_handler(self, message: str, thread_id: str, message_bus: MessageBus):
        """异步处理器"""
        response = {"status": "success", "message": f"收到消息: {message}"}
        message_bus.publish(thread_id, response)
        return response

service = MyService()
resp = service.call(message="测试消息")
print(list(resp))

[{'status': 'success', 'message': '收到消息: 测试消息', 'topic': 'test_service.2af975d8-d3c8-11ef-abbd-acde48001122'}, {'block_type': 'end', 'topic': 'test_service.2af975d8-d3c8-11ef-abbd-acde48001122'}]


In [8]:
resp = await service.async_call(message="hi")
async for x in resp:
    print(x)

{'status': 'success', 'message': '收到消息: hi', 'topic': 'test_service.3116e7e8-d3c8-11ef-abbd-acde48001122'}
{'block_type': 'end', 'topic': 'test_service.3116e7e8-d3c8-11ef-abbd-acde48001122'}
